# Deep Neural Network - Basic

In [ ]:
def prepare_total_data_nn(datas):
    data_sim_sign = datas[1].sample(frac=1).reset_index(drop=True)
    data_bkg = datas[0].sample(frac=1).reset_index(drop=True)
    data_sig_train = data_sim_sign
    data_bkg_train = data_bkg[:len(data_sig_train)]
    data_train = [data_sig_train, data_bkg_train]
#    data_sig_test = data_sim_sign[int(train_size*len(data_sim_sign)):]
#    data_bkg_test = data_bkg[int(train_size*len(data_bkg)):]
#    data_test = data_sig_test.append(data_bkg_test[:len(data_sig_test)],
#                                     ignore_index=True).sample(frac=1).reset_index(drop=True)
#    x_test = data_test.iloc[:,:-1]
#    y_test = data_test.iloc[:,-1]
#    x_test.to_csv('Data_test/x_test_nn_ensemble.csv', mode='w+')
#    y_test.to_csv('Data_test/y_test_nn_ensemble.csv', mode='w+')
    return data_train

In [ ]:
def train_val_split(x_train, y_train):
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    list_k = list(skf.split(x_train_nn, y_train_nn))
    return list_k

In [ ]:
def get_kfold_data(x_train, y_train, fold):
    X_train = x_train[fold[0]]
    Y_train = y_train[fold[0]]
    X_val = x_train[fold[1]]
    Y_val = y_train[fold[1]]
    return X_train, Y_train, X_val, Y_val

In [ ]:
def show_nn_evolution(tr_acc, tr_loss, val_acc, val_loss):
    epochs = range(1, len(tr_acc)+1)
    fig = plt.figure(figsize=(20,7))
    
    ax_acc = plt.subplot(1,2,1)
    ax_acc.plot(epochs, tr_acc, 'ro', label='Training accuracy')
    ax_acc.plot(epochs, val_acc, 'g', label='Validation accuracy')
    ax_acc.set_title('Training and validation accuracy')
    ax_acc.set_xlabel('Epochs')
    ax_acc.set_ylabel('Accuracy')
    ax_acc.legend()
    
    ax_loss = plt.subplot(1,2,2)
    ax_loss.plot(epochs, tr_loss, marker='o', color='orange', label='Training loss')
    ax_loss.plot(epochs, val_loss, 'b', label='Validation loss')
    ax_loss.set_title('Training and validation loss')
    ax_loss.set_xlabel('Epochs')
    ax_loss.set_ylabel('Loss')
    ax_loss.legend()
    return

# Deep Neural Network - Advanced

In [ ]:
def build_model(input_shape, layers_size=[30,30], batch_normalization=True, dropout=None):
    model = models.Sequential()
    model.add(layers.Dense(layers_size[0], activation='relu', input_shape=(input_shape,)))
    if batch_normalization:
        model.add(layers.BatchNormalization())
    for layer_size in layers_size[1:]:
        model.add(layers.Dense(layer_size, activation='relu'))
        if dropout!=None:
            model.add(layers.Dropout(dropout))
        if batch_normalization:
            model.add(layers.BatchNormalization())
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def evaluate_nn(x_train, y_train, epochs=50, layers_size=[30,30], batch_normalization=True,
                dropout=None, batch_size=10, verbose=0, return_max_val_acc=False, show_graph=True):
    k_folds_indx = train_val_split(x_train_nn, y_train_nn)

    tr_acc, tr_loss, val_acc, val_loss = [], [], [], []
    
    for i,fold in enumerate(k_folds_indx):
        print('Processing', i+1, '/', len(k_folds_indx), 'fold.')
        X_train, Y_train, X_val, Y_val = get_kfold_data(x_train_nn, y_train_nn, fold)
        model = build_model(input_shape=x_train.shape[1],
                            layers_size=layers_size,
                            batch_normalization=batch_normalization,
                           dropout=dropout)
        history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,
                            validation_data=(X_val, Y_val), verbose=verbose)
        history_dict = history.history
        tr_acc.append(history_dict['accuracy'])
        tr_loss.append(history_dict['loss'])
        val_acc.append(history_dict['val_accuracy'])
        val_loss.append(history_dict['val_loss'])
    tr_acc = np.mean(tr_acc, axis=0)
    tr_loss = np.mean(tr_loss, axis=0)
    val_acc = np.mean(val_acc, axis=0)
    val_loss = np.mean(val_loss, axis=0)
    max_val_acc = np.max(val_acc)
    if show_graph:
        print("The maximum validation accuracy reached is", max_val_acc)
        show_nn_evolution(tr_acc, tr_loss, val_acc, val_loss)
    if return_max_val_acc:
        return max_val_acc
    return

In [ ]:
def search_for_batch_size(x_train, y_train, epochs=50, layers_size=[30,30], batch_normalization=True,
                        dropout=None, verbose=0):
    history_max_val_acc = []
    batch_size_try = list(range(1,4,1))
    for batch_size in batch_size_try:
        print("Processing batch size", batch_size)
        if batch_size<10:
            epochs = 35
        max_val_acc = evaluate_nn(x_train_nn, y_train_nn, epochs=epochs,
                                layers_size=layers_size,
                                batch_normalization=batch_normalization,
                                dropout=dropout,
                                batch_size=batch_size,
                                verbose=0,
                                return_max_val_acc=True,
                                show_graph=False)
        history_max_val_acc.append(max_val_acc)
    plt.plot(batch_size_try, history_max_val_acc)
    return

In [ ]:
def print_pars(layer_size, batch_norm, dropout):
    print('Evaluating nn configuration:\nLayer size:\t', layer_size, '\nBatch normalization:\t', batch_norm,
         '\nDropout:\t', dropout)
    return

In [ ]:
def try_nn_parameters(parameters):
    fig = plt.figure(figsize=(25,25))
    indx = 0
    for parameter in parameters:
        layers_size = [25,25]
        batch_normalization = True
        dropout = 0.45
        result = []
        epochs = 130
        batch_size = 80
        if parameter=='layers_size':
            layers_size_try = parameters[parameter]
            for layers_size in layers_size_try:
                print_pars(layers_size, batch_normalization, dropout)
                max_val_acc = evaluate_nn(x_train_nn, y_train_nn, epochs=epochs,
                                layers_size=layers_size,
                                batch_normalization=batch_normalization,
                                dropout=dropout,
                                batch_size=batch_size,
                                verbose=0,
                                return_max_val_acc=True,
                                show_graph=False)
                result.append(max_val_acc)
            indx+=1
            ax = plt.subplot(math.ceil(len(parameters)/2),2,indx)
            ax.set_title(parameter)
            ax.plot(range(len(result)), result)
            
        if parameter=='batch_normalization':
            batch_normalization_try = parameters[parameter]
            for batch_normalization in batch_normalization_try:
                print_pars(layers_size, batch_normalization, dropout)
                max_val_acc = evaluate_nn(x_train_nn, y_train_nn, epochs=epochs,
                                layers_size=layers_size,
                                batch_normalization=batch_normalization,
                                dropout=dropout,
                                batch_size=batch_size,
                                verbose=0,
                                return_max_val_acc=True,
                                show_graph=False)
                result.append(max_val_acc)
            indx+=1
            ax = plt.subplot(math.ceil(len(parameters)/2),2,indx)
            ax.set_title(parameter)
            ax.plot(range(len(result)), result)
            
        if parameter=='dropout':
            dropout_try = parameters[parameter]
            for dropout in dropout_try:
                print_pars(layers_size, batch_normalization, dropout)
                max_val_acc = evaluate_nn(x_train_nn, y_train_nn, epochs=epochs,
                                layers_size=layers_size,
                                batch_normalization=batch_normalization,
                                dropout=dropout,
                                batch_size=batch_size,
                                verbose=0,
                                return_max_val_acc=True,
                                show_graph=False)
                result.append(max_val_acc)
            indx+=1
            ax = plt.subplot(math.ceil(len(parameters)/2),2,indx)
            ax.set_title(parameter)
            ax.plot(dropout_try, result)
    
    return
            
        

# Deep Neural Network - Create and train final

In [ ]:
def make_all_nn_models(dropouts, batch_normalizations, layers_sizes, input_shape,
                      best_dropout, best_batch_normalization, best_layers_sizes, with_weights=True):
    models = []
    weights = []
    for dropout in dropouts:
        for batch_normalization in batch_normalizations:
            for layers_size in layers_sizes:
                model = build_model(input_shape=input_shape,
                                    layers_size=layers_size,
                                    batch_normalization=batch_normalization,
                                    dropout=dropout)
                models.append(model)
                if (dropout==best_dropout) and (batch_normalization==best_batch_normalization):
                    if layers_size in best_layers_sizes:
                        weights.append(2)
                    else:
                        weights.append(1)
                else:
                    weights.append(1)
    if with_weights:
        return models, weights
    else:
        return models

In [ ]:
def get_data_for_fit(data_train, counter):
    data_sig = data_train[0]
    data_bkg = data_train[1].sample(frac=1).reset_index(drop=True)[:len(data_sig)]
    data_tot = data_sig.append(data_bkg, ignore_index=True).sample(frac=1).reset_index(drop=True)
    X_train = data_tot.iloc[:,:-1]
    Y_train = data_tot.iloc[:,-1]
    scaler = StandardScaler()
    scaler.fit(X_train, Y_train)
    scaler_path = os.path.join('Scalers', 'std_scaler_' + str(counter) + '.bin')
    joblib.dump(scaler, scaler_path, compress=True)
    return X_train, Y_train, scaler, scaler_path

In [ ]:
def get_val_data_for_fit(data_train):
    data_sig = data_train[0].sample(frac=1).reset_index(drop=True)
    data_bkg = data_train[1].sample(frac=1).reset_index(drop=True)
    data_sig_val = data_sig[:int(0.3*len(data_sig))]
    data_bkg_val = data_bkg[:len(data_sig_val)]
    data_val = data_sig_val.append(data_bkg_val, ignore_index=True).sample(frac=1).reset_index(drop=True)
    X_val = data_val.iloc[:,:-1]
    Y_val = data_val.iloc[:,-1]
    data_remaining = [data_sig[int(0.3*len(data_sig)):], data_bkg[len(data_sig_val):]]
    return data_remaining, X_val, Y_val

In [ ]:
def prepare_data_nn(x_train, y_train, x_test, y_test):
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    y_train = np.asarray(y_train, dtype='float32')
    x_test = scaler.transform(x_test)
    y_test = np.asarray(y_test, dtype='float32')
    return x_train, y_train, x_test, y_test

In [ ]:
def prepare_data_nn_ensemble(x_train, y_train, x_val, y_val, scaler):
    x_train = scaler.transform(x_train)
    y_train = np.asarray(y_train, dtype='float32')
    x_val = scaler.transform(x_val)
    y_val = np.asarray(y_val, dtype='float32')
    return x_train, y_train, x_val, y_val

In [ ]:
def fit_nn_models(models, data_train, batch_size, epochs, just_best_models=True):
    
    counter = 1
    ckpt_paths = []
    scalers_paths = []
    
    if just_best_models:
        start_file_name = 'best_model_'
    else:
        start_file_name = 'model_'
    
    for model in models:
        
        data_training, X_val, Y_val = get_val_data_for_fit(data_train)
        print('Processing', counter, '/', len(models), 'model.')
        
        X_train, Y_train, scaler, scaler_path = get_data_for_fit(data_training, counter)
        scalers_paths.append(scaler_path)
        
        X_train, Y_train, X_val, Y_val = prepare_data_nn_ensemble(X_train, Y_train, X_val, Y_val, scaler)
        
        ckpt_name = start_file_name + str(counter) + '.ckpt'
        ckpt_path = os.path.join('Models_checkpoints', ckpt_name)
        checkpoint = ModelCheckpoint(ckpt_path, monitor='val_accuracy', verbose=1,
                                     save_best_only=True, mode='max')
        callbacks_list = [checkpoint]
        model.fit(X_train, Y_train,
                  epochs=epochs,
                  batch_size=batch_size,
                  callbacks=callbacks_list,
                  validation_data=(X_val, Y_val),
                  verbose=0
                 )
        
        ckpt_paths.append(ckpt_path)
        counter += 1
    return models, ckpt_paths, scalers_paths

In [ ]:
def write_checkpoints_to_file(checkpoints, all_models=True):
    if all_models:
        file_path = 'all_models_checkpoints.txt'
    else:
        file_path = 'best_models_checkpoints.txt'
    with open(file_path, 'w+') as f:
        for checkpoint in checkpoints:
            f.write(checkpoint + '\n')

In [ ]:
def write_scalers_to_file(scalers, all_models=True):
    if all_models:
        file_path = 'all_models_scalers.txt'
    else:
        file_path = 'best_models_scalers.txt'
    with open(file_path, 'w+') as f:
        for scaler in scalers:
            f.write(scaler + '\n')

# Deep Neural Network - Predicting

In [ ]:
def fetch_test_data_ensemble_nn():
    x_train = pd.read_csv('Data_test/x_test_nn_ensemble.csv', index_col=0, sep=",")
    y_train = pd.read_csv('Data_test/y_test_nn_ensemble.csv', index_col=0, sep=",", header=None, names=['Label'])
    return x_train, y_train

In [ ]:
def fetch_models(models, all_models=True):
    if all_models:
        checkpoints_path = 'all_models_checkpoints.txt'
        scalers_path = 'all_models_scalers.txt'
    else:
        checkpoints_path = 'best_models_checkpoints.txt'
        scalers_path = 'best_models_scalers.txt'
        
    with open(checkpoints_path, 'r') as f:
        checkpoints = f.readlines()
    checkpoints = [checkpoint[:-1] for checkpoint in checkpoints]
    models_loaded = []
    for model, checkpoint in zip(models, checkpoints):
        model.load_weights(checkpoint)
        models_loaded.append(model)
        
    with open(scalers_path, 'r') as f:
        scalers = f.readlines()
    scalers = [scaler[:-1] for scaler in scalers]
    scalers_loaded = []
    for scaler in scalers:
        scaler_loaded = joblib.load(scaler)
        scalers_loaded.append(scaler_loaded)
        
    return models_loaded, scalers_loaded

In [ ]:
class Ensemble_NN():
    
    def __init__(self, models, scalers, weights=None):
        self.models = models
        self.scalers = scalers
        self.weights = weights
        
    def predict(self, x_train):
        votes_proba = np.zeros((x_train.shape[0],1))
        for model, scaler in zip(self.models, self.scalers):
            x_train_model = scaler.transform(x_train)
            vote = model.predict(x_train_model)
            votes_proba += np.array(vote)
        votes_proba /= len(self.models)
        return np.round(votes_proba).astype(int)
    
    def predict_proba(self, x_train):
        votes_proba = np.zeros((x_train.shape[0],1))
        for model, scaler in zip(self.models, self.scalers):
            x_train_model = scaler.transform(x_train)
            vote = model.predict(x_train_model)
            votes_proba += np.array(vote)
        votes_proba /= len(self.models)
        return votes_proba